In [9]:
import nltk

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

from nltk import word_tokenize

import pandas as pd 

import re
from nltk.corpus import stopwords
import nltk

In [12]:
CovidAllTweets = list(Covide_data['OriginalTweet'].apply(word_tokenize))

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
Covide_data = pd.read_csv('/content/drive/MyDrive/hadi/Covid_19_tweets_train.csv',encoding="latin1")
#Covide_data=Covide_data.dropna()
Covide_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier,RidgeClassifier,MultiTaskLasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import NuSVC

In [55]:
stop_words=stopwords.words('english')

def tweets_vanish(all_tweet):
    tweets=all_tweet.map(lambda x :x.lower())
    tweets=tweets.map(lambda x:re.sub(r'([^a-z])|(http\S+)',' ',x))
    tweets =tweets.map(lambda tweet: ' '.join([word for word in tweet.lower().split() if word not in stop_words and len(word)>1]))
    return tweets

In [56]:
Cleaning_tweets = tweets_vanish(Covide_data['OriginalTweet'])

In [57]:
Covid_train, Covid_test, y_train, y_test = train_test_split(Cleaning_tweets, Covide_data['Sentiment'], test_size=0.25, random_state=1000)

In [58]:
y_train

18698              Negative
26797    Extremely Negative
37956              Positive
18927    Extremely Positive
34119               Neutral
                ...        
33537    Extremely Positive
18782              Negative
3776     Extremely Negative
6215     Extremely Positive
4695     Extremely Negative
Name: Sentiment, Length: 30867, dtype: object

In [59]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.preprocessing.text import Tokenizer

In [60]:
import gensim
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import scipy
from gensim.test.utils import common_texts
import numpy as np

In [61]:
types=['Negative','Extremely Negative','Extremely Positive','Neutral','Positive']

In [62]:
types_one_hot_encoding=np.zeros((5, 5))

for i in range(len(types)):
    types_one_hot_encoding[i,i]=1

In [63]:
dic_type_one_hot_encoding={}
for i in range(len(types)):
    dic_type_one_hot_encoding[types[i]]=types_one_hot_encoding[i]

In [64]:
indexing_types={}
for i in range(len(types)):
    indexing_types[types[i]]=i

In [65]:
def get_feature_from_one_xtrain(sample):
    result=[]
    summ=0
    
    for word in sample.split():
#         print(word)
        summ+=1
        try: 
#             feature=np.sum(model.wv[word])
            result.append(model.wv[word])
        except:
            feature=0
        if summ==50:
            return result
    return result

In [66]:
def get_feature_from_all_xtrain(samples):
    res=[]
    for sample in samples:
        s=get_feature_from_one_xtrain(sample)

        if(len(s)<50):
            
            for i in range(50-len(s)):
                x=np.zeros(100,dtype=np.float64)
                s.append(x)
            
#         x=np.zeros(20,dtype=np.float64)
#         for i in range(20):
#             if(i <len(s)):
                
#                 x[i]=s[i]
        res.append(s)
    return res

In [67]:
# sentences=Cleaning_tweets
# sentences=pd.Series(sentences)
# sentences=sentences.apply(lambda x:str(x))
# sentences=sentences.apply(lambda x:x.split(" "))

In [68]:
# model = Word2Vec(sentences)

In [69]:
# X_Train=get_feature_from_all_xtrain(Covid_train)
Y_Train=[dic_type_one_hot_encoding[i] for i in y_train]

In [70]:
Y_Test=[dic_type_one_hot_encoding[i] for i in y_test]

In [71]:
Y_Test=np.array(Y_Test)
Y_Train=np.array(Y_Train)

In [72]:

print(Y_Test.shape)
print(Y_Train.shape)

(10290, 5)
(30867, 5)


In [73]:
# model_train=Sequential()
# model_train.add(Bidirectional(LSTM(units=128, input_shape=(50,100), activation="relu")))
# model_train.add(Dense(units=10, activation='tanh'))
# model_train.add(Dense(units=28, activation='relu'))
# model_train.add(Dense(units=5, activation='softmax'))
# model_train.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['acc'])

In [74]:
# model_train.fit(X_Train,Y_Train,epochs=20,validation_split=0.2,callbacks=[EarlyStopping('val_loss',patience=0)],batch_size=256)

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


In [76]:
maxlen=100
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(Covid_train)
X_train = tokenizer.texts_to_sequences(Covid_train)
X_test = tokenizer.texts_to_sequences(Covid_test)

    # Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

    # Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [77]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [78]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[10000], 
                  embedding_dim=[50,100,300],
                  maxlen=[100])

In [79]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [80]:
model = KerasClassifier(build_fn=create_model,
                            epochs=4, batch_size=16,
                            verbose=True)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                           cv=4, verbose=1, n_iter=8)
grid_result = grid.fit(X_train, Y_Train)
#  # Evaluate testing set
test_accuracy = grid.score(X_test, Y_test)

Fitting 4 folds for each of 8 candidates, totalling 32 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/4
1447/1447 [==============================] - 30s 19ms/step - loss: 1.3848 - accuracy: 0.3916
Epoch 2/4
1447/1447 [==============================] - 28s 19ms/step - loss: 0.7691 - accuracy: 0.7247
Epoch 3/4
1447/1447 [==============================] - 27s 19ms/step - loss: 0.6278 - accuracy: 0.7791
Epoch 4/4
483/483 [==============================] - 4s 6ms/step - loss: 0.8816 - accuracy: 0.7033
Epoch 1/4
1447/1447 [==============================] - 30s 19ms/step - loss: 1.4322 - accuracy: 0.3513
Epoch 2/4
1447/1447 [==============================] - 27s 19ms/step - loss: 0.7623 - accuracy: 0.7264
Epoch 3/4
1447/1447 [==============================] - 27s 19ms/step - loss: 0.6036 - accuracy: 0.7925
Epoch 4/4
483/483 [==============================] - 4s 6ms/step - loss: 0.9392 - accuracy: 0.6877
Epoch 1/4
1447/1447 [==============================] - 29s 18ms/step - loss: 1.3805 - accuracy: 0.3892
Epoch 2/4
1447/1447 [==============================] - 27s 18ms/step - loss: 0.75

KeyboardInterrupt: ignored